In [ ]:
# Data processing  
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import pickle

# Pandas options  
# -----------------------------------------------------------------------
pd.options.display.max_colwidth = None

# Path configuration for custom module imports  
# -----------------------------------------------------------------------
import sys  
sys.path.append('../')  # Adds the parent directory to the path for custom module imports  

# Ignore warnings  
# -----------------------------------------------------------------------
import warnings  
warnings.filterwarnings("ignore") 

# Machine learning imports
# -----------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler

# Custom functions and classes
# -----------------------------------------------------------------------
from src.support_handling import impute_knn
from src.support_encoding import Encoding

## Data loading

In [7]:
df = pd.read_csv('../data/output/complete_data.csv', index_col=0).reset_index(drop=True)

In [8]:
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,11,0,1.0,6,1,0,3.0,4.0,2.0,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,...,23,1,6.0,3,5,1,3.0,2.0,4.0,2
2,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,...,15,3,5.0,2,5,0,2.0,2.0,1.0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,...,11,3,13.0,5,8,7,4.0,4.0,3.0,2
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,...,12,2,9.0,2,6,0,4.0,1.0,3.0,3


## Missing values management

In [9]:
missing_values = ((df.isnull().sum() / df.shape[0]) * 100).round(2)

missing_values[missing_values > 0]

NumCompaniesWorked         0.43
TotalWorkingYears          0.20
EnvironmentSatisfaction    0.57
JobSatisfaction            0.45
WorkLifeBalance            0.86
dtype: float64

Tenemos menos de 1% de nulos en algunas columnas. Vamos a imputarlos utilizando k-nearest-neighbours

In [10]:
df_imp = impute_knn(df, 'Attrition', n=5)

In [11]:
# Vemos que hemos imputado los nulos de las numéricas
df_imp.isnull().sum()

Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
JobInvolvement             0
dtype: int64

## Outlier imputing

De momento no lo hacemos

## Encoding

In [ ]:
# StandardScaler
numeric_features = ['size', 'rooms', 'distance', 'numPhotos']
numeric_transformer = StandardScaler()

scaled_data = numeric_transformer.fit_transform(df[numeric_features])
df[numeric_features] = scaled_data